# 1. Preparación de los datos

In [20]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.


In [21]:
# Importación de las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from unidecode import unidecode

In [15]:
# Cargamos el archivo CSV
df = pd.read_csv('../datasets/items_titles.csv')

In [16]:
# Mostramos el tamaño del dataset 
print("Tamaño del dataset")
print(f"Filas: {df.shape[0]}")
print(f"Columnas: {df.shape[1]}")

# Vemos el tipo de datos
print("\nInformacion general del dataset")
df.info()

#Mostramos algunas filas del dataset
df

Tamaño del dataset
Filas: 10000
Columnas: 1

Informacion general del dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ITE_ITEM_TITLE  10000 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde
...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil
9996,Sapatenis Casual Masculino Estiloso 24horas Co...
9997,Tênis Feminino Infantil Molekinha Tie Dye
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...


Podemos observar que el dataset contiene una columna llamada `ITE_ITEM_TITLE` con los títulos de productos, que vamos a usar para medir similitud textual.

In [25]:
#Renombramos la columna para mayor comodidad
df_clean = df.copy()
df_clean = df_clean.rename(columns={'ITE_ITEM_TITLE': 'title'})

# Función de limpieza de texto actualizada
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()  # Pasar a minúsculas
    text = unidecode(text)  # Eliminar acentos (tênis -> tenis)
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar signos de puntuación
    text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios duplicados
    return text

# Aplicamos la función al dataset
df_clean['clean_title'] = df_clean['title'].apply(clean_text)

#Mostramos una muestra de antes y después
df_clean[['title', 'clean_title']].head(10)

,title,clean_title
0,Tênis Olympikus Esporte Valente - Masculino Kids,tenis olympikus esporte valente masculino kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,bicicleta barra forte samy c 6 marchas cubo c ...
2,Tênis Usthemp Slip-on Temático - Labrador 2,tenis usthemp slipon tematico labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye,tenis casual feminino moleca tecido tie dye
4,Tênis Star Baby Sapatinho Conforto + Brinde,tenis star baby sapatinho conforto brinde
5,Tênis Oakley Frequency 3.0 Preto/marrom,tenis oakley frequency 30 pretomarrom
6,Tênis Jogging Feminino Premium Super Lançament...,tenis jogging feminino premium super lancament...
7,Under Armour Hovr Phantom 2 Conexão Bluetooth ...,under armour hovr phantom 2 conexao bluetooth ...
8,Tenis Infantil Feminino Menina Criança Moça,tenis infantil feminino menina crianca moca
9,Tênis Labellamafia Saturn 6 Cores Disponíveis,tenis labellamafia saturn 6 cores disponiveis


Relicé una limpieza adicional (normalización) eliminando los acentos y caracteres especiales utilizando la librería unidecode. Ya que los títulos están en portugués (ej: "tênis", "camiseta", etc.) y pueden contener variaciones ortográficas que afectan la comparación entre cadenas.

Remover los acentos permite mejorar la calidad de las comparaciones, ya que palabras como "tênis" y "tenis" (que se refieren al mismo producto) pasan a ser tratadas como equivalentes.

# 2. Vectorización